### $\textit{Muitas pessoas ouvem recomendações e conselhos, mas só os prudentes e coerentes se beneficiam deles.}$

### $\textbf{Sistemas recomendarores}$

Os sistemas de recomendação buscam simplificar a busca por informações relevantes em meio a vastos conjuntos de dados, proporcionando aos usuários uma experiência mais personalizada e eficiente. Ao entender as preferências individuais, esses sistemas contribuem significativamente para a usabilidade e satisfação do usuário em ambientes online.

Como base sejá utilizado o conjunto de dados user_interests.

In [34]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

A recomendação de novos interesses com bases nos seus interesses específicos também será aborado.

### $\textbf{Recomendando as opções mais populares:}$

In [51]:
from collections import Counter

popular_interests = Counter(interest 
                            for user_interests in users_interests 
                            for interest in user_interests)

In [37]:
from typing import Dict, Tuple

def most_popular_new_interests(user_interests: list[str], max_results: int = 5) -> list[Tuple[str, int]]:
    suggestions = [(interest, frequency) 
                   for interest, frequency in popular_interests.most_common() 
                   if interest not in user_interests]
    return suggestions[:max_results]

In [38]:
# Para um usuário com os seguintes interesses temos as seguintes recomendações
usuario_1 = ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
print(most_popular_new_interests(usuario_1))

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]


### $\textbf{Filtragem Colaborativa baseada no usuário:}$

Para se procurar usuários semelhantes, usaremos a semelhança entre dois cossenos de vetores de 0's e 1's. Cada vetor $v$ representará os interesses de um usuário:

- $v[i]$ será 1 se o usuário especificar o interesse $i$
- 0 caso contrário

Assim, usuários semelhantes terão vetores que apontam na mesma direção. Usuário com semelhança igual a 1 são iguais, com semelhança 0 são completamentes diferentes. Outros pares de usuários terão valores entre 0 e 1.

In [39]:
unique_interests = sorted({interest
                           for user_interests in users_interests
                           for interest in user_interests})

In [40]:
def make_user_interest_vector(user_interests: list[str]) -> list[int]:
    return [1 if interest in user_interests 
            else 0 
            for interest in unique_interests]

In [41]:
user_interest_vectors = [make_user_interest_vector(user_interests) for user_interests in users_interests]

Iteramos sobre os interesses de todos os usuários e calculamos suas similaridades. De forma que:

\begin{equation*}
    \text{cosine\_similarity}(v_1,v_2) = \frac{v_1 \cdot v_2}{||v_{1} \cdot v_{2}||}
\end{equation*}


In [43]:
import math

def dot(v, w) -> float:
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def cosine_similarity(v1, v2) -> float:
    return dot(v1, v2) / math.sqrt(dot(v1, v1) * dot(v2, v2))

user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_vectors]
                      for interest_vector_i in user_interest_vectors]

In [48]:
def most_similar_users_to(user_id: int) -> list[Tuple[int, float]]:
    pairs = [(other_user_id, similarity)                      # Find other
             for other_user_id, similarity in                 # users with
                enumerate(user_similarities[user_id])         # nonzero
             if user_id != other_user_id and similarity > 0]  # similarity.

    return sorted(pairs,                                      # Sort them
                  key=lambda pair: pair[-1],                  # most similar
                  reverse=True)                               # first.

Usuários mais similares ao usuário 0

In [75]:
# Usuários similares a n
most_similar_users_to(0)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

Utilizando estes valores para recomendar para um usuário. 

In [52]:
from collections import defaultdict

def user_based_suggestions(user_id: int,
                           include_current_interests: bool = False):
    # Sum up the similarities.
    suggestions: Dict[str, float] = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    # Convert them to a sorted list.
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[-1],  # weight
                         reverse=True)

    # And (maybe) exclude already-interests
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [74]:
# SUgestão para o n-ésimo usuário
user_based_suggestions(0)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

### $\textbf{Filtragem Colaborativa baseada em itens:}$

Uma alternativa é computar diretamente a semelhança entre os interesses. Para começar, é preciso transport a matriz interesse-usuário, dessa forma as linhas vão corresponder aos interesses e as colunas ao usuários:

In [60]:
interest_user_matrix = [[user_interest_vector[j]
                        for user_interest_vector in user_interest_vectors]
                        for j, _ in enumerate(unique_interests)]

Para este caso, se dois usuários estiverem interesse em um mesmo tópico a semelhança será 1 e, se não estiverm, a semelhança será 0.

In [61]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                          for user_vector_j in interest_user_matrix]
                          for user_vector_i in interest_user_matrix]

Para encontrar o interesse mais próximo de 'big data':

In [65]:
def most_similar_interests_to(interest_id: int):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs, key=lambda pair: pair[-1], reverse=True)

In [72]:
#Interesses similares ao n-ésimo interesse
most_similar_interests_to(0)

[('Hadoop', 0.8164965809277261),
 ('Java', 0.6666666666666666),
 ('MapReduce', 0.5773502691896258),
 ('Spark', 0.5773502691896258),
 ('Storm', 0.5773502691896258),
 ('Cassandra', 0.4082482904638631),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('HBase', 0.3333333333333333)]

Recomendações para o usuário somando as semelhanças dos interesses parecidos com os dele:

In [68]:
def item_based_suggestions(user_id: int,
                           include_current_interests: bool = False):
    # Some os interesses semelhantes
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_vectors[user_id]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    # Organize-os pelo peso
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[-1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [71]:
#Para o usuário n
item_based_suggestions(0)

[('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]